In [2]:
import pandas as pd
import numpy as np

In [3]:
# %run -i "../solve_jax.py" -m KH -L1 2 
#* you can run the python executable directly and can use the same arguments defined in the executable

In [110]:
spins = [2] * 12
groundstate = np.load("out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
groundstate = groundstate.reshape(spins)
x = groundstate.copy()

## Check if the transition symmetry is correct

In [80]:
r1 = x.reshape(2**3, -1)
r2 = x.transpose([6,7,8,9,10,11,0,1,2,3,4,5]).reshape(2**3, -1)
partial_trace1 = np.einsum("im,jm->ij", r1, r1)
partial_trace2 = np.einsum("im,jm->ij", r2, r2)
print("difference of partial trace : ", np.linalg.norm(partial_trace1 - partial_trace2))
rho = partial_trace1
rho = (rho + rho.T)/2 #* make sure rho is Hermitian ( moreover, it should be positive semi-definite )

difference of partial trace :  1.4910943e-07


## Conduct eigen decomposition

In [91]:
E, V = np.linalg.eigh(rho)
print("sum of eigenvalues = ",E.sum()) #* should be 1
print("All eigenvalues are positive : ", (E>=0).all())
print("recreate from eigenvalues and vectors / diff : ",np.linalg.norm(V @ np.diag(E) @ V.T - rho)) #* should be zero

sum of eigenvalues =  1.0000002
All eigenvalues are positive :  True
recreate from eigenvalues and vectors / diff :  3.5450057e-08


### Check overwrap with local ground state

- #### Partical Trace for n = 6,7,8,9,10,11


In [111]:
x = groundstate.copy()
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
print("check trace : ", rho.trace())

check trace :  1.0000002


#### Original Hamiltonian


H1 is local hamiltonian between triangle (0,1,2) and (3,4,5)


In [130]:
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
E0, V0 = np.linalg.eigh(H0)
E1, V1 = np.linalg.eigh(H1)
E2, V2 = np.linalg.eigh(H2)
for i in range(20):
    print(f"{i}th local ground state" ,V1[:,i] @ rho @ V1[:,i]) 

#### Check overwarpping with stochastic local ground state with local ground state

In [145]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
import nsp
SH1 = -nsp.utils.stoquastic(-H1) #* make all-off-diagonal elements negative
SE1, SV1 = np.linalg.eigh(SH1)

since stoquastic local ground state can be all positive vector, and try to be the same vector up to sign of its elements, inner product of the two vectors can be good measure of optimization.

- Of course, you can compare it with their ground energy.

In [149]:
print(np.abs(SV1[:,0]) @ np.abs(V1[:,0]) )
print(abs(SE1[0]-E1[0]))

0.985526456763024
0.026504056397945175


This is really good result, which means local hamiltonian almost doesn't have negative sign. 
Next compare for Partical Diagonalized Hamiltonian

In [158]:
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3siteDiag/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
E0, V0 = np.linalg.eigh(H0)
E1, V1 = np.linalg.eigh(H1)
E2, V2 = np.linalg.eigh(H2)

### Optimized Hamiltonian

In [218]:
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/2.npy")
u = np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/u/0.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]

# E0, V0 = np.linalg.eigh(H0)
# E1, V1 = np.linalg.eigh(H1)
# E2, V2 = np.linalg.eigh(H2)

# SE, SV = np.linalg.eigh(SH_list[1])

for l in range(3):
    E, V = np.linalg.eigh(H_list[l])
    SE, SV = np.linalg.eigh(SH_list[l])
    print(f"local {l}th")
    print(np.abs(SV[:,0]) @ np.abs(V[:,0]), abs(SE[0]-E[0]))

local 0th
0.9999349078484887 0.00034916060574463614
local 1th
0.9999283544467124 0.0002446148999630804
local 2th
0.7072735320822696 0.0017438369396829412


Check if the Local Hamiltonian is expressed by unitary matrix

In [190]:
OH0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
OE0, OV0 = np.linalg.eigh(OH0)
np.linalg.norm(np.kron(u,u) @ H0 @ np.kron(u,u).T - OH0)

6.213945649735919e-14

# Servey with Exact Diagonalization

In [1]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import nsp
from jax_exact.core.constants import *
import numpy as np

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test original Hamiltonian

In [89]:

# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [90]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)



In [91]:
i = 0
x = _V[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

x = _V[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

x = _V[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
rho = np.einsum("im,jm->ij", r, r)
rho = (rho + rho.T)/2
LE, LV = np.linalg.eigh(H_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LV[:,i] @ rho @ LV[:,i])

l = 0 / innder product of eigenvector [0] :  0.20229731150902824
l = 1 / innder product of eigenvector [0] :  0.20229698950424807
l = 2 / innder product of eigenvector [0] :  0.20229675550945114


The above show that global ground state is covered by local ground state to some extent.

#### Stoquastic hamiltonian

In [92]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [93]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state -0.8145067


In [94]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.32738108943819233
local stoquastic energy of ground state : -1.0265040563979462
--------------------
l = 1 / innder product of eigenvector [0] :  0.32738109690556505
local stoquastic energy of ground state : -1.026504056397945
--------------------
l = 2 / innder product of eigenvector [0] :  0.32738110380336116
local stoquastic energy of ground state : -1.0265040563979455
--------------------


In [105]:
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
for i in range(5):
    print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
    print(f"local stoquastic energy of ground state : {LSE[i]}")




l = 1 / innder product of eigenvector [0] :  0.32738109690556505
local stoquastic energy of ground state : -1.026504056397945
l = 1 / innder product of eigenvector [1] :  0.15025257200947267
local stoquastic energy of ground state : -0.8793846160070569
l = 1 / innder product of eigenvector [2] :  0.15025264442891278
local stoquastic energy of ground state : -0.879384616007056
l = 1 / innder product of eigenvector [3] :  0.00827017081088954
local stoquastic energy of ground state : -0.8460056123187534
l = 1 / innder product of eigenvector [4] :  0.014376807409863935
local stoquastic energy of ground state : -0.8460056123187528
-0.6829196213414735


#### Calculate energy using density matrix

In [22]:
_SHT = nsp.utils.sum_ham(SH_list[0], [_bonds[0][0]], 4, 8)
_SHT = jnp.array(_SHT)
e_test = _SV[:,0] @ _SHT @ _SV[:,0]


e_sum = 0
for i in range(len(LSV[0])):
    print(f"dot = {LSV[:,i] @ Srho @ LSV[:,i]:3}")
    e_sum += LSV[:,i] @ Srho @ LSV[:,i] * LSE[i]
print("energy calculated by density matrix: ", e_sum)
print("energy calculated by ground state: ", e_test)

dot = 0.14017802137647267
dot = 0.3758865247444629
dot = 0.3474908743306805
dot = 0.12130706114810602
dot = 3.226729698284084e-05
dot = 8.189981614850446e-05
dot = 7.798991502110583e-05
dot = 2.577637071557837e-05
dot = 2.3867006075274005e-08
dot = 0.006745428810205151
dot = 0.007145650944577893
dot = 1.5355275037047434e-08
dot = 3.7869982796859167e-10
dot = 2.106688153391306e-06
dot = 9.276090589524561e-07
dot = 1.0738835031307937e-09
dot = 4.089329283792569e-06
dot = 2.3502667160431978e-05
dot = 3.914500700378338e-05
dot = 4.190841307701314e-05
dot = 7.742224838582336e-05
dot = 7.982941151935945e-05
dot = 3.634818649108459e-05
dot = 1.986628406152843e-08
dot = 1.3815201037499042e-05
dot = -1.2919297383518137e-09
dot = 3.6450314497419304e-08
dot = 5.934891434320779e-07
dot = 9.636831807068233e-07
dot = 7.501752730280998e-06
dot = 1.326211423717509e-09
dot = -1.1275237781648826e-09
dot = 4.512586699636711e-10
dot = -4.3744523353788594e-10
dot = 7.576761638863526e-07
dot = 5.42579802702

In [21]:
LSE[:10]

array([-1.00024461, -0.97903667, -0.95421481, -0.92284767, -0.8480168 ,
       -0.82741435, -0.80358326, -0.77404457, -0.75023699, -0.71900751])

### Test optimized Hamiltonian

In [1]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import nsp
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Pure hamiltonian

In [2]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [3]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [4]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state 0.4446661


Check overlapping of stoquastic local ground state with stoquastic global ground state

In [5]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.13976965871283176
local stoquastic energy of ground state : -1.000349160605744
--------------------
l = 1 / innder product of eigenvector [0] :  0.14017802137647267
local stoquastic energy of ground state : -1.000244614899966
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984060123397892
local stoquastic energy of ground state : -1.0017438369396832
--------------------


This suggests that the wors optimized local hamiltonian amoung three behave badly, leading huge gap between global ground state energy?

In [18]:
_SHT = nsp.utils.sum_ham(SH_list[1], [_bonds[1][0]], 4, 8)
_SHT = jnp.array(_SHT)
e_test = _SV[:,0] @ _SHT @ _SV[:,0]

e_sum = 0
x = _SV[:,0].copy()
l = 1
LSE, LSV = np.linalg.eigh(SH_list[l])
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
for i in range(len(LSV[0])):
    print(f"dot = {LSV[:,i] @ Srho @ LSV[:,i]:.3}")
    e_sum += LSV[:,i] @ Srho @ LSV[:,i] * LSE[i]
print("energy calculated by density matrix: ", e_sum)
print("energy calculated by ground state: ", e_test)

dot = 0.14
dot = 0.376
dot = 0.347
dot = 0.121
dot = 3.23e-05
dot = 8.19e-05
dot = 7.8e-05
dot = 2.58e-05
dot = 2.39e-08
dot = 0.00675
dot = 0.00715
dot = 1.54e-08
dot = 3.79e-10
dot = 2.11e-06
dot = 9.28e-07
dot = 1.07e-09
dot = 4.09e-06
dot = 2.35e-05
dot = 3.91e-05
dot = 4.19e-05
dot = 7.74e-05
dot = 7.98e-05
dot = 3.63e-05
dot = 1.99e-08
dot = 1.38e-05
dot = -1.29e-09
dot = 3.65e-08
dot = 5.93e-07
dot = 9.64e-07
dot = 7.5e-06
dot = 1.33e-09
dot = -1.13e-09
dot = 4.51e-10
dot = -4.37e-10
dot = 7.58e-07
dot = 5.43e-06
dot = 1.8e-06
dot = 2.81e-07
dot = 1.88e-09
dot = 1.55e-07
dot = -6.71e-11
dot = 3.81e-10
dot = 9.96e-05
dot = 8.13e-05
dot = 0.000111
dot = 4.7e-05
dot = 0.000188
dot = -4.45e-10
dot = 1.69e-10
dot = -2.34e-10
dot = 9.68e-10
dot = 6.54e-07
dot = 2.42e-06
dot = 1.31e-09
dot = 3.6e-11
dot = 2.34e-10
dot = 3.2e-10
dot = 1.27e-09
dot = -3.14e-10
dot = 1.07e-09
dot = 4.91e-10
dot = 7.74e-05
dot = 8.42e-05
dot = 4.15e-10
energy calculated by density matrix:  -0.9614801185863

This suggests that second and thrid eigen values are too bit and they greatly contribute to global gs energy

### Test optimized Hamiltonian in modified scheme (take minimum of three)

In [64]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_244/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [65]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [61]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [62]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))
_SE

Inner product of ground state -0.62793386


Array([-11.4248295, -11.421765 ,  -9.034054 , ...,   5.4970393,
         5.9149704,   5.9155645], dtype=float32)

In [63]:
for l in range(3):
    print(np.linalg.eigvalsh(SH_list[l])[0])

-1.000177700541588
-1.158498533022129
-1.0000676290110921


Check overlapping of stoquastic local ground state with stoquastic global ground state

In [55]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.13976965871283176
local stoquastic energy of ground state : -1.000349160605744
--------------------
l = 1 / innder product of eigenvector [0] :  0.14017802137647267
local stoquastic energy of ground state : -1.000244614899966
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984060123397892
local stoquastic energy of ground state : -1.0017438369396832
--------------------


In this case, all hamiltonian optimized well. But only one of them occupy the global ground state. I expected it equally distributed among three.

### Test optimized Hamiltonian in modified scheme (take maximum of three)

In [67]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
from jax_exact.core.constants import *
import numpy as np
# minus is for compenstating the minus sign in the stores local Hamiltonian
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/H/2.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]




bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])


#### Pure hamiltonian

In [68]:
_H = nsp.utils.sum_ham(H_list[0], _bonds[0], 4, 8)
_H += nsp.utils.sum_ham(H_list[1], _bonds[1], 4, 8)
_H += nsp.utils.sum_ham(H_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_H = jnp.array(_H)
_E, _V = jax.scipy.linalg.eigh(_H)


#### Stoquastic hamiltonian

In [69]:
_SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
_SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)

from jax import numpy as jnp
import jax
_SH = jnp.array(_SH)
_SE, _SV = jax.scipy.linalg.eigh(_SH)

In [73]:
print("local stoquastic energy of ground state are ")
for l in range(3):
    print(np.linalg.eigvalsh(SH_list[l])[:5])

local stoquastic energy of ground state are 
[-1.00024887 -0.97956155 -0.95481491 -0.92328368 -0.84801807]
[-1.00042168 -0.97964041 -0.95470461 -0.92293865 -0.84783306]
[-1.00393964 -0.99921168 -0.84927691 -0.84916852 -0.84760019]


In [72]:
print(_SE[:5])
print(_E[:5])

[-11.716789 -11.672066  -8.9985    -8.971427  -8.839621]
[-5.4448752 -5.328395  -5.2982364 -5.2982364 -5.165295 ]


In [39]:
print("Inner product of ground state", _SV[:,0] @ jnp.abs(_V[:,0]))

Inner product of ground state 0.4446772


Note that the result is worser than the one with optimization of average.
Check overlapping of stoquastic local ground state with stoquastic global ground state

In [41]:
i = 0
x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,6,7,8,3,4,5,9,10,11]) # for l = 0
l = 0
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 0 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)

x = _SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
print(f"l = 2 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
print(f"local stoquastic energy of ground state : {LSE[0]}")
print("-"*20)


l = 0 / innder product of eigenvector [0] :  0.1486917598370786
local stoquastic energy of ground state : -1.000248865011823
--------------------
l = 1 / innder product of eigenvector [0] :  0.14748093107524332
local stoquastic energy of ground state : -1.0004216832046362
--------------------
l = 2 / innder product of eigenvector [0] :  0.9984103447999974
local stoquastic energy of ground state : -1.0039396389784765
--------------------


In this case, all hamiltonian optimized well. But only one of them occupy the global ground state. I expected it equally distributed among three.

In [44]:
x = _SV[:,0].copy()
l = 1
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])
for i in range(10):
    print(f"l = 1 / innder product of eigenvector [{i}] : ",LSV[:,i] @ Srho @ LSV[:,i])
    print(f"local stoquastic energy of ground state : {LSE[i]}")

l = 1 / innder product of eigenvector [0] :  0.14748093107524332
local stoquastic energy of ground state : -1.0004216832046362
l = 1 / innder product of eigenvector [1] :  0.39957868359682625
local stoquastic energy of ground state : -0.9796404081959669
l = 1 / innder product of eigenvector [2] :  0.3223953965597398
local stoquastic energy of ground state : -0.9547046096658286
l = 1 / innder product of eigenvector [3] :  0.11561606151367278
local stoquastic energy of ground state : -0.922938647596009
l = 1 / innder product of eigenvector [4] :  3.311896112410876e-05
local stoquastic energy of ground state : -0.8478330584558101
l = 1 / innder product of eigenvector [5] :  8.254510672067024e-05
local stoquastic energy of ground state : -0.8276318741800783
l = 1 / innder product of eigenvector [6] :  7.667640706862021e-05
local stoquastic energy of ground state : -0.8038576974393363
l = 1 / innder product of eigenvector [7] :  2.490896607239086e-05
local stoquastic energy of ground state 

This suggest global ground state is mostly occupied by excited state rather than ground state.

# Comparison of two cases : 1) original Hamiltonian, 2) optimized Hamiltonian

In [27]:
import sys
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import nsp
from jax_exact.core.constants import *
import numpy as np

from jax import numpy as jnp
import jax

bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
_bonds = [[], [], []]
for bond in bonds:
    _bonds[bond[0]].append(bond[1:])

In [31]:
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
u = np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/u/0.npy")

H_list = [H0, H1, H2]
SH_list = [
    -nsp.utils.stoquastic(-H) for H in H_list
]

In [45]:
H0 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/0.npy")
H1 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/1.npy")
H2 = -np.load("/home/user/project/python/make_local/array/KH/3site/mes/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_240/H/2.npy")

OH_list = [H0, H1, H2] #* optimized Hamiltonian
OSH_list = [
    -nsp.utils.stoquastic(-H) for H in OH_list
]

In [60]:
SH = nsp.utils.sum_ham(SH_list[0], _bonds[0], 4, 8)
SH += nsp.utils.sum_ham(SH_list[1], _bonds[1], 4, 8)
SH += nsp.utils.sum_ham(SH_list[2], _bonds[2], 4, 8)
SH = jnp.array(SH)
SE, SV = jax.scipy.linalg.eigh(SH)


OSH = nsp.utils.sum_ham(OSH_list[0], _bonds[0], 4, 8)
OSH += nsp.utils.sum_ham(OSH_list[1], _bonds[1], 4, 8)
OSH += nsp.utils.sum_ham(OSH_list[2], _bonds[2], 4, 8)
OSH = jnp.array(OSH)
OSE, OSV = jax.scipy.linalg.eigh(OSH)


In [48]:
print("gs energy of stoquastic Hamiltonian : ", SE[0])
print("gs energy of optimized stoquastic Hamiltonian : ", OSE[0])

gs energy of stoquastic Hamiltonian :  -8.195041
gs energy of optimized stoquastic Hamiltonian :  -11.695035


This is huge difference although the optimized Hamiltonian give a good result in terms of local ground state.

In [51]:
for l in range(3):
    LSE = np.linalg.eigvalsh(SH_list[l])
    LOSE = np.linalg.eigvalsh(OH_list[l])
    print(f"l = {l} / local stoquastic ene spectrum           = {np.round(LSE[:5],3)}", )
    print(f"l = {l} / local optimized stoquastic ene spectrum = {np.round(LOSE[:5],3)}", )

l = 0 / local stoquastic ene spectrum           = [-1.027 -0.879 -0.879 -0.846 -0.846]
l = 0 / local optimized stoquastic ene spectrum = [-1.    -0.848 -0.848 -0.848 -0.848]
l = 1 / local stoquastic ene spectrum           = [-1.027 -0.879 -0.879 -0.846 -0.846]
l = 1 / local optimized stoquastic ene spectrum = [-1.    -0.848 -0.848 -0.848 -0.848]
l = 2 / local stoquastic ene spectrum           = [-1.027 -0.879 -0.879 -0.846 -0.846]
l = 2 / local optimized stoquastic ene spectrum = [-1.    -0.848 -0.848 -0.848 -0.848]


It looks like just improving gs energy without any flaw.
But as a matter of fact, and as you already confirmed in the above cells, the global ground eneryg is rather high.

### hamiltonian index = 1

In [96]:

x = SV[:,0].copy()
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])

x = OSV[:,0].copy()
r = x.reshape(2**6, -1)
OSrho = np.einsum("im,jm->ij", r, r)
OSrho = (OSrho + OSrho.T)/2
LOSE, LOSV = np.linalg.eigh(OSH_list[l])

ene_sum = 0
opt_ene_sum = 0
for i in range(len(LSE)):
    ene_sum += LSE[i] * (LSV[:,i] @ Srho @ LSV[:,i])
    opt_ene_sum += LOSE[i] * (LOSV[:,i] @ OSrho @ LOSV[:,i])
    if (i < 7):
        # print(f"                energy : {ene_sum:.3} / optimized : {opt_ene_sum:.3}")
        print(f"i={i} original / similarity : {LSV[:,i] @ Srho @ LSV[:,i]:.3} / ene : {LSE[i]:.3}/ ene_sum : {ene_sum:.3}")
        print(f"   optimized / similarity : {LOSV[:,i] @ OSrho @ LOSV[:,i]:.3} / ene : {LOSE[i]:.3}/ ene_sum : {opt_ene_sum:.3}")
                            
        print("")
print("exp energy of single local hamiltonian: ", ene_sum)
print("exp energy of optimized local hamiltonian: ", opt_ene_sum)

i=0 original / similarity : 0.327 / ene : -1.03/ ene_sum : -0.336
   optimized / similarity : 0.14 / ene : -1.0/ ene_sum : -0.14

i=1 original / similarity : 0.15 / ene : -0.879/ ene_sum : -0.468
   optimized / similarity : 0.376 / ene : -0.979/ ene_sum : -0.508

i=2 original / similarity : 0.15 / ene : -0.879/ ene_sum : -0.6
   optimized / similarity : 0.347 / ene : -0.954/ ene_sum : -0.84

i=3 original / similarity : 0.00827 / ene : -0.846/ ene_sum : -0.607
   optimized / similarity : 0.121 / ene : -0.923/ ene_sum : -0.952

i=4 original / similarity : 0.0144 / ene : -0.846/ ene_sum : -0.619
   optimized / similarity : 3.23e-05 / ene : -0.848/ ene_sum : -0.952

i=5 original / similarity : 0.00207 / ene : -0.845/ ene_sum : -0.621
   optimized / similarity : 8.19e-05 / ene : -0.827/ ene_sum : -0.952

i=6 original / similarity : 0.00207 / ene : -0.845/ ene_sum : -0.623
   optimized / similarity : 7.8e-05 / ene : -0.804/ ene_sum : -0.952

exp energy of single local hamiltonian:  -0.682919

### Hamiltonian index = 2

In [99]:

l = 2
x = SV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
r = x.reshape(2**6, -1)
Srho = np.einsum("im,jm->ij", r, r)
Srho = (Srho + Srho.T)/2
LSE, LSV = np.linalg.eigh(SH_list[l])

x = OSV[:,0].copy()
x = x.reshape([2]*12)
x = x.transpose([0,1,2,9,10,11,3,4,5,6,7,8]) # for l = 2
r = x.reshape(2**6, -1)
OSrho = np.einsum("im,jm->ij", r, r)
OSrho = (OSrho + OSrho.T)/2
LOSE, LOSV = np.linalg.eigh(OSH_list[l])

ene_sum = 0
opt_ene_sum = 0
for i in range(len(LSE)):
    ene_sum += LSE[i] * (LSV[:,i] @ Srho @ LSV[:,i])
    opt_ene_sum += LOSE[i] * (LOSV[:,i] @ OSrho @ LOSV[:,i])
    if (i < 7):
        # print(f"                energy : {ene_sum:.3} / optimized : {opt_ene_sum:.3}")
        print(f"i={i} original / similarity : {LSV[:,i] @ Srho @ LSV[:,i]:.3} / ene : {LSE[i]:.3}/ ene_sum : {ene_sum:.3}")
        print(f"   optimized / similarity : {LOSV[:,i] @ OSrho @ LOSV[:,i]:.3} / ene : {LOSE[i]:.3}/ ene_sum : {opt_ene_sum:.3}")
                            
        print("")
print("exp energy of single local hamiltonian: ", ene_sum)
print("exp energy of optimized local hamiltonian: ", opt_ene_sum)

i=0 original / similarity : 0.327 / ene : -1.03/ ene_sum : -0.336
   optimized / similarity : 0.998 / ene : -1.0/ ene_sum : -1.0

i=1 original / similarity : 0.15 / ene : -0.879/ ene_sum : -0.468
   optimized / similarity : 6.84e-07 / ene : -1.0/ ene_sum : -1.0

i=2 original / similarity : 0.15 / ene : -0.879/ ene_sum : -0.6
   optimized / similarity : 5.29e-08 / ene : -0.849/ ene_sum : -1.0

i=3 original / similarity : 0.0113 / ene : -0.846/ ene_sum : -0.61
   optimized / similarity : 0.000893 / ene : -0.849/ ene_sum : -1.0

i=4 original / similarity : 0.0113 / ene : -0.846/ ene_sum : -0.619
   optimized / similarity : 6.01e-08 / ene : -0.848/ ene_sum : -1.0

i=5 original / similarity : 0.00207 / ene : -0.845/ ene_sum : -0.621
   optimized / similarity : 3.4e-08 / ene : -0.848/ ene_sum : -1.0

i=6 original / similarity : 0.00207 / ene : -0.845/ ene_sum : -0.623
   optimized / similarity : 5.34e-07 / ene : -0.728/ ene_sum : -1.0

exp energy of single local hamiltonian:  -0.682919604204

Above result shows exitation states largely contribute to global ground state energy. (only ground state contributes for l=2)